### Imports

In [1]:
import gurobipy as gp
import pandas as pd

### File Input

In [2]:
df = pd.read_excel("Nutritional Facts.xlsx", index_col=0)

### Restaurant / Category Selection

In [3]:
res = "Arby's"
cat = "Sandwiches"
subset = df[df["Restaurant"] == res][df["Category"] == cat]
subset  # Contains all Arby's Sandwiches

C:\Users\zacha\Anaconda3\envs\NutritionResearch\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,Restaurant,Category,Food,Serving Size,Calories From Fat,Calories,Total Fat,Saturated Fat,Trans Fat,Cholesterol,...,Protein,Total Fat %,Saturated Fat %,Cholesterol %,Sodium %,Total Carbohydrates %,Dietary Fiber %,Protein %,Vitamin A %,URL
0,Arby's,Sandwiches,Arby's Melt,146g,110,330,12g,4.0g,.5g,35mg,...,18g,18%,20%,12%,39%,13%,8%,36%,2%,https://fastfoodnutrition.org/arbys/arbys-melt
1,Arby's,Sandwiches,Arby-Q Sandwich,182g,100,400,11g,3.5g,.5g,30mg,...,18g,17%,18%,10%,52%,19%,12%,36%,4%,https://fastfoodnutrition.org/arbys/arby-q-san...
2,Arby's,Sandwiches,Beef 'n Cheddar Classic,1 sandwich,180,450,20g,6g,1g,50mg,...,23g,31%,30%,17%,53%,15%,8%,46%,2%,https://fastfoodnutrition.org/arbys/beef-n-che...
3,Arby's,Sandwiches,Beer Battered Fish Sandwich,1 sandwich,210,600,23g,4g,0g,65mg,...,25g,35%,20%,22%,76%,24%,20%,50%,NaN,https://fastfoodnutrition.org/arbys/beer-batte...
4,Arby's,Sandwiches,Buffalo Crispy Chicken Sandwich,1 sandwich,210,500,23g,4.5g,0.0g,55mg,...,24g,35%,23%,18%,78%,16%,16%,48%,NaN,https://fastfoodnutrition.org/arbys/buttermilk...
5,Arby's,Sandwiches,Buffalo Roast Chicken Sandwich,1 sandwich,130,360,14g,3.5g,0g,65mg,...,24g,22%,18%,22%,62%,12%,12%,48%,NaN,https://fastfoodnutrition.org/arbys/buffalo-ro...
6,Arby's,Sandwiches,Chicken Bacon Swiss Sandwich,1 sandwich,270,610,30g,9.0g,0.0g,85mg,...,35g,46%,45%,28%,63%,17%,20%,70%,NaN,https://fastfoodnutrition.org/arbys/buttermilk...
7,Arby's,Sandwiches,Chicken Cheddar Ranch Sandwich,1 sandwich,270,610,30g,8g,0g,65mg,...,43g,46%,40%,22%,58%,19%,16%,86%,NaN,https://fastfoodnutrition.org/arbys/chicken-ch...
8,Arby's,Sandwiches,Chicken Cordon Bleu – Roast,230g,190,500,22g,6.0g,0.0g,85mg,...,35g,34%,30%,28%,66%,13%,8%,70%,2%,https://fastfoodnutrition.org/arbys/chicken-co...
9,Arby's,Sandwiches,"Chicken Tenders, 2-piece",2 tenders,110,240,12g,2g,0g,40mg,...,15g,18%,10%,13%,33%,6%,4%,30%,0%,https://fastfoodnutrition.org/arbys/prime-cut-...


### Sample Constraint Setting

In [5]:
protein_req = 20      # g  (50)
cholesterol_lim = 50  # mg (100)

### Model Building

In [6]:
m = gp.Model()

# Variables
xis = [m.addVar(vtype=gp.GRB.BINARY) for _ in subset]

# Constraints
m.addConstr(sum([x * float(subset.iloc[r]["Protein"].strip("g")) for r, x in enumerate(xis)]) >= protein_req)
m.addConstr(sum([x * float(subset.iloc[r]["Cholesterol"].strip("mg")) for r, x in enumerate(xis)]) <= cholesterol_lim)

# Objective
m.ModelSense = gp.GRB.MINIMIZE
m.setObjective(sum([x * int(subset.iloc[r]["Calories From Fat"].strip("g")) for r, x in enumerate(xis)]))

Using license file C:\Apps\Gurobi\gurobi.lic
Academic license - for non-commercial use only


### Solving

In [7]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 2 rows, 24 columns and 48 nonzeros
Model fingerprint: 0x8a286681
Variable types: 0 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [2e+01, 2e+02]
  Objective range  [1e+02, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 5e+01]
Presolve removed 2 rows and 24 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 120 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.200000000000e+02, best bound 1.200000000000e+02, gap 0.0000%


### Optimal Objective

In [9]:
print(m.ObjVal, "calories from fat")
m.printAttr('X')
best_choices = [i for i, x in enumerate(xis) if xis[i].x == 1]

120.0 calories from fat

    Variable            X 
-------------------------
         C14            1 


### Corresponding Meal(s)

In [11]:
print(subset.iloc[best_choices]["Food"], ":")
subset.iloc[best_choices]

14    Classic Roast Beef
Name: Food, dtype: object :


,Restaurant,Category,Food,Serving Size,Calories From Fat,Calories,Total Fat,Saturated Fat,Trans Fat,Cholesterol,...,Protein,Total Fat %,Saturated Fat %,Cholesterol %,Sodium %,Total Carbohydrates %,Dietary Fiber %,Protein %,Vitamin A %,URL
14,Arby's,Sandwiches,Classic Roast Beef,154g,120,360,14g,5.0g,0.5g,50mg,...,23g,22%,25%,17%,40%,12%,8%,46%,0%,https://fastfoodnutrition.org/arbys/classic-ro...
